## Agentic Workflows in LlamaIndex

A workflow in LlamaIndex provides a structured way to organize your code into sequential and manageable steps.

`Workflow`s strike a balance between the autonomy of agents while maintaining control of the overall workflow.

## Basic Workflow creation

In [4]:
from llama_index.core.workflow import StartEvent, StopEvent, Workflow, step

class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent)-> StopEvent:
        # do something here
        return StopEvent(result='Hello World!')
    

w = MyWorkflow(timeout=10, verbose=True)
result = await w.run()
result

Running step my_step
Step my_step produced event StopEvent


'Hello World!'

### Connecting multiple steps
Connect multiple steps through custom events that carry data between steps. 

In [5]:
from llama_index.core.workflow import Event

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent)-> ProcessingEvent:
        # Process initial data
        return ProcessingEvent(intermediate_result='Step 1 processed')
    
    @step
    async def step_two(self, ev: ProcessingEvent)-> StopEvent:
        # Use the intermediate result
        final_result = f'Finished Processing: {ev.intermediate_result}'
        return StopEvent(result=final_result)
    
w = MultiStepWorkflow(timeout=10, verbose=True)
result = await w.run()
result

Running step step_one
Step step_one produced event ProcessingEvent
Running step step_two
Step step_two produced event StopEvent


'Finished Processing: Step 1 processed'

### Loops and Branches

`Type hinting` is the most powerful part of workflows because it allows us to create branches, loops and joins to facilitate more complex workflows.

In [6]:
from tabnanny import verbose
from llama_index.core.workflow import Event
import random

class ProcessingEvent(Event):
    intermediate_result: str

class LoopEvent(Event):
    loop_output: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> ProcessingEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print('Bad thing happened')
            return LoopEvent(loop_output='Back to step 1')

        else:
            print('Good thing happened')
            return ProcessingEvent(intermediate_result='First step complete')

    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # Use the intermediate result
        final_result = f'Finished processing: {ev.intermediate_result}'
        return StopEvent(result=final_result)

w = MultiStepWorkflow(timeout=10, verbose=True)
result = await w.run()
result
        
        

        

Running step step_one
Good thing happened
Step step_one produced event ProcessingEvent
Running step step_two
Step step_two produced event StopEvent


'Finished processing: First step complete'

### Drawing Workflows


In [8]:
from llama_index.utils.workflow import draw_all_possible_flows
draw_all_possible_flows(w, 'flow.html')

flow.html


Gtk-Message: 21:06:26.865: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


### State Management

This uses the `Context` object we saw in the `AgentsInLlamaIndex` Notebook.

In [9]:
from llama_index.core.workflow import Context, StartEvent, StopEvent, Event
from llama_index.core.agent.workflow import ReActAgent

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent, ctx: Context) -> ProcessingEvent:
        # process initial data:
        await ctx.set('query', 'What is the capital of France?')
        return ProcessingEvent(intermediate_result='Step 1 complete')

    @step
    async def step_two(self, ev: ProcessingEvent, ctx: Context) -> StopEvent:
        # Use the intermediate result
        query = await ctx.get('query')
        print(f'Query: {query}')
        final_result = f'Finished Processing: {ev.intermediate_result}'
        return StopEvent(result=final_result)
    
w = MultiStepWorkflow(timeout=10, verbose=True)
result = await w.run()
result

Running step step_one
Step step_one produced event ProcessingEvent
Running step step_two
Query: What is the capital of France?
Step step_two produced event StopEvent


'Finished Processing: Step 1 complete'

## Multi-Agent Workflows

Instead of manual `workflow` creation, we can use the `AgentWorkflow` class to create a multi-agent workflow. 
The `AgentWorkflow` uses `Workflow` Agents to allow you to create a system of one or more agents that can collaborate and hand off tasks to each other based on their specialized capabilities, allowing for complex agent systems where different agents handle different aspects of a task. 
One agent must be designated as the root agent in the `AgentWorkflow` constructor. When a user message comes in, it is first routed to the root agent.

Each agent can then:
- Handle the request directly using their tools
- Handoff to another agent better suited for the task
- Return a response to the user


In the below system, we also add `Context` to show context sharing (as a simple usecase of keeping track of the number of function calls)

In [15]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from llama_index.core.workflow import Context
from llama_index.llms.openai import OpenAI

async def add(ctx: Context, a: int, b: int) -> int:
    """Add two numbers."""
    # update count
    cur_state = await ctx.get('state')
    cur_state['num_fn_calls'] += 1
    await ctx.set('state', cur_state)
    
    return a + b

async def multiply(ctx: Context, a: int, b: int) -> int:
    """Multiply two numbers."""

    # update count
    cur_state = await ctx.get('state')
    cur_state['num_fn_calls'] += 1
    await ctx.set('state', cur_state)
    
    return a * b

from dotenv import load_dotenv
load_dotenv()
llm = OpenAI('gpt-4o-mini')

multiply_agent = ReActAgent(
    name='multiply_agent',
    description='Is able to multiple two numbers',
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools = [multiply],
    llm=llm
)

addition_agent = ReActAgent(
    name='addition_agent',
    description='Is able to add two integers',
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools = [add],
    llm=llm
)

# create the workflow
workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent='multiply_agent',
    initial_state={'num_fn_calls': 0},
    state_prompt='Current state: {state}. User Message: {msg}'
)

# Run the system, with context
ctx = Context(workflow)
response = await workflow.run(user_msg = 'Can you add 5 and 3?', ctx=ctx)
print(f'Response: {response}')

# pull out and inspect state
state = await ctx.get('state')
print(f'State: {state["num_fn_calls"]}')


Response: The sum of 5 and 3 is 8.
State: 1
